# 5.5 Case study: resonant freqencies of a micro-resonant transducer

This case study uses the beam element introdced in Chapter 5 to model a micro-resonant transducer. The addition over the single-element derivation is that multiple beams are used, but since the geometry is still in the X-Y plane and the beams are not rotated, the only thing to do is to
a) add more elements
b) assemble the global stiffness, and mass matrices utilizing the same techniques as in Chapter 4, e.g. Example 4.2.


In [1]:
# number of elements to be used. The book uses 10.
NE = 10  # number of elements

# number of degrees of freedom each node
ND = 2  # 2D problem, so only X and Y coordinates are used

# number of degrees of freedom per element. This is not to be changed.
DOF = 4  # u_x, u_y, u_z, θ_y, θ_z


In [ ]:
# imports
import numpy as np
from source.OneD.node import Node
from source.OneD.beam.beam import BeamElement
np.set_printoptions(precision=2, threshold=3, suppress=False)

### Geometry and material properties

Defined as in the book and converted to SI units for consistency.

In [3]:
b = 20  # width of the beam [µm]
h = 0.5  # height of the beam [µm]
L = 100  # length of the beam [µm]
E = 169  # Young's modulus [GPa]
nu = 0.262  # Poisson's ratio
ro = 2.30  # density [g/cm³]

# transforming everything in SI units: meters, pascals, kilograms
b *= 1e-6  # width in meters
h *= 1e-6  # height in meters
L *= 1e-6  # length in meters
E *= 1e9  # Young's modulus in pascals
ro *= 1e3  # density in kg/m³

### Cross sectional properties

In [4]:
A = b * h  # cross-sectional area [m²]
Iy = b * h**3 / 12  # second moment of area around the y-axis [m^4]
Iz = h * b**3 / 12  # second moment of area around the z-axis [m^4]

### FE mesh and supports.

In [5]:
# Mesh
# The total length of the beam is divided into NE elements, each element has a length of L/NE.
mesh = np.linspace(0, L, NE + 1)  # mesh points along the beam

# Create nodes based on the mesh
nodes = [Node(x, 0, None) for x in mesh]  # nodes in the X-Y plane, Z=0

print()
print("Nodes:")
for node in nodes:
    print(f"Node ID: {node.ID}, Coordinates: ({node.x}, {node.y})")

# Create beam elements. The inertia to be used is the second moment of area around the y-axis.
elements = [
    BeamElement(nodes[i], nodes[i + 1], A=A, I=Iy, E=E, ro=ro)
    for i in range(NE)
]
print()
print("Elements:")
for elem in elements:
    print(f"Element from Node {elem.i.ID} to Node {elem.j.ID}, Area: {elem.A}, Iy: {elem.I}, E: {elem.E}, ro: {elem.ro}")

# Convert nodes and elements to dictionaries for easy access.
# This is required for the assembly functions later.
nodes = {n.ID: n for n in nodes}
elements = {e.ID: e for e in elements}

# clamped supports are at both ends.
supports = (0, (0, 1),  # (node ID, (dof1, dof2))
                  NE, (0, 1))  # (node ID, (dof1, dof2))
print(supports)



Nodes:
Node ID: 0, Coordinates: (0.0, 0)
Node ID: 1, Coordinates: (9.999999999999999e-06, 0)
Node ID: 2, Coordinates: (1.9999999999999998e-05, 0)
Node ID: 3, Coordinates: (2.9999999999999997e-05, 0)
Node ID: 4, Coordinates: (3.9999999999999996e-05, 0)
Node ID: 5, Coordinates: (4.9999999999999996e-05, 0)
Node ID: 6, Coordinates: (5.9999999999999995e-05, 0)
Node ID: 7, Coordinates: (7e-05, 0)
Node ID: 8, Coordinates: (7.999999999999999e-05, 0)
Node ID: 9, Coordinates: (8.999999999999999e-05, 0)
Node ID: 10, Coordinates: (9.999999999999999e-05, 0)

Elements:
Element from Node 0 to Node 1, Area: 1e-11, Iy: 2.083333333333333e-25, E: 169000000000.0, ro: 2300.0
Element from Node 1 to Node 2, Area: 1e-11, Iy: 2.083333333333333e-25, E: 169000000000.0, ro: 2300.0
Element from Node 2 to Node 3, Area: 1e-11, Iy: 2.083333333333333e-25, E: 169000000000.0, ro: 2300.0
Element from Node 3 to Node 4, Area: 1e-11, Iy: 2.083333333333333e-25, E: 169000000000.0, ro: 2300.0
Element from Node 4 to Node 5, Ar

### Global force vector

In [6]:
F = np.zeros(ND * len(nodes))  # Global force vector, initialized to zero
F[ND * (NE // 2)] = -1e-3  # Apply a force of -1 mN in the y-direction at the middle node
print(F)
print([x for x in F if x != 0])  # Print only non-zero entries

[0. 0. 0. ... 0. 0. 0.]
[np.float64(-0.001)]


### Assembling the global stiffness and mass matrices

In [7]:


K_global = assemble_global_K(ND=ND, nodes=nodes, elements=elements)
K_global, F = apply_boundary_conditions(ND=ND, supports=supports, K=K_global, F=F)

def assemble_M():
    pass

ImportError: cannot import name 'assemble_global_K' from 'source.utils' (E:\modules\FEM_Practical_Course\source\utils.py)